In [2]:
import pandas as pd
import matplotlib as plt

In [3]:
transactions = pd.read_csv("./data/amazon-purchases.csv")

In [4]:
print(transactions.head())

   Order Date  Purchase Price Per Unit  Quantity Shipping Address State  \
0  2018-12-04                     7.98       1.0                     NJ   
1  2018-12-22                    13.99       1.0                     NJ   
2  2018-12-24                     8.99       1.0                     NJ   
3  2018-12-25                    10.45       1.0                     NJ   
4  2018-12-25                    10.00       1.0                     NJ   

                                               Title ASIN/ISBN (Product Code)  \
0  SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...               B0143RTB1E   
1  Betron BS10 Earphones Wired Headphones in Ear ...               B01MA1MJ6H   
2                                                NaN               B078JZTFN3   
3  Perfecto Stainless Steel Shaving Bowl. Durable...               B06XWF9HML   
4                      Proraso Shaving Cream for Men               B00837ZOI0   

        Category  Survey ResponseID  
0   FLASH_MEMORY  R_01vN

In [5]:
transactions["Order Date"] = pd.to_datetime(transactions["Order Date"])

In [6]:
transactions_sorted = transactions.sort_values(["Survey ResponseID", "Order Date"], ascending=[True, True])

## Adding customer information

In [7]:
customers = pd.read_csv("./data/survey.csv")

### Formating Living state to match with Temperature Table
We use the living state of customer table to fill in missing values in the transaction table -> need to be in the same format

In [8]:
# Create a dictionary of state name mappings
state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Assuming your dataframe is called 'df' and the column with state names is called 'state'
customers["Q-demos-state"] = customers["Q-demos-state"].replace(state_abbrev)

### Mergin Dataframes

In [9]:
amazon_data = transactions.merge(customers, on="Survey ResponseID", how="left")

### Dealing with missing shipping addresses

Most of missing values in "Shipping Address State" can be filled by looking at "Q-demos-state" from the customer dataset -> we asume that the residence state is the same state as the shipping state 

In [10]:
amazon_data.loc[amazon_data["Shipping Address State"].isna(), "Shipping Address State"] = amazon_data["Q-demos-state"] # Replacing missing Shipping Addresses with living address from ordering customer

amazon_data.loc[amazon_data["Shipping Address State"] == "PR", "Shipping Address State"] = amazon_data["Q-demos-state"] # There is a shipping address state called PR which could be Puerto Rico, we replace this with the living state of the customer ordering

## Adding Temperature information

In [11]:
weather = pd.read_csv("./Data/weather_data.csv")

print(weather["st_abb"].nunique()) # No weather information for Alaska and Hawaii
weather.head()

48


,Unnamed: 0,st_abb,st_code,date,stability,ppt,tmin,tmax,tavg
0,1,AL,1,20180101,stable,1.602,-7.569,3.303,-2.133
1,2,AL,1,20180102,stable,0.002,-9.746,-0.868,-5.307
2,3,AL,1,20180103,stable,0.000,-9.822,2.161,-3.830
3,4,AL,1,20180104,stable,0.000,-5.802,4.918,-0.442
4,5,AL,1,20180105,stable,0.013,-8.308,2.945,-2.681


In [12]:
weather["date"] = pd.to_datetime(weather["date"], format="%Y%m%d")
weather.rename(columns={"st_abb": "Shipping Address State"}, inplace=True)
weather.head()

,Unnamed: 0,Shipping Address State,st_code,date,stability,ppt,tmin,tmax,tavg
0,1,AL,1,2018-01-01,stable,1.602,-7.569,3.303,-2.133
1,2,AL,1,2018-01-02,stable,0.002,-9.746,-0.868,-5.307
2,3,AL,1,2018-01-03,stable,0.000,-9.822,2.161,-3.830
3,4,AL,1,2018-01-04,stable,0.000,-5.802,4.918,-0.442
4,5,AL,1,2018-01-05,stable,0.013,-8.308,2.945,-2.681


In [13]:
# Joining weather

data_combined = pd.merge(
    left=amazon_data, 
    right=weather,
    how='left',
    left_on=['Shipping Address State', 'Order Date'],
    right_on=['Shipping Address State', 'date'],
)

In [14]:
# Check for which states we have no temperature information
# AK, DC, HI, I did not reside in the United States
# We still leave them in the dataset for completnes 

data_grouped = data_combined.groupby("Shipping Address State")
data_grouped["tavg"].count()

Shipping Address State
AK                                            0
AL                                        18830
AR                                        14859
AZ                                        34273
CA                                       187520
CO                                        30697
CT                                        18021
DC                                            0
DE                                         7723
FL                                       115704
GA                                        56123
HI                                            0
I did not reside in the United States         0
IA                                        11441
ID                                         6528
IL                                        76102
IN                                        46960
KS                                        16386
KY                                        28466
LA                                        20973
MA               

## CHECKPOINT Saving complete combined dataset

In [15]:
data_combined.to_csv('./data/data_combined.csv')